In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import os
import matplotlib.pyplot as plt
import io

from collections import defaultdict
import torch
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist

from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.graphs import Neo4jGraph
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI



from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget


from dotenv import load_dotenv
load_dotenv()

/usr/local/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
vrf_data_path = "../../full_data/vrf_data.xlsx"
seva_data_path = "../../full_data/seva_modified2.xlsx"

vrf_df = pd.read_excel(vrf_data_path)
candidate_info_df = pd.read_excel(seva_data_path)
vrf_jobs_df = vrf_df['/'].dropna().drop_duplicates().reset_index(drop=True)#[:10]
corpus_str = ",\n".join(vrf_jobs_df)

# docs = [Document(page_content=corpus_str)]

# # Step 3: Initialize the RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=40, chunk_overlap=10)

# # Step 4: Split the document
# documents = text_splitter.split_documents(documents=docs)
documents = [
    Document(page_content=corpus_str)
]

In [54]:
openai_api_key = os.getenv("OPENAI_API_KEY")

llm_type = 'chat_gpt'
if llm_type == 'chat_gpt':
    model = "gpt-4o" # 'gpt-4o-mini'
    llm = ChatOpenAI(model=model, api_key=openai_api_key)
elif llm_type == 'llama':
    llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(
    llm=llm)
    # allowed_nodes=["Job"],
    # allowed_relationships=["PARENT_TO", "RELATED_TO", "SIMILAR_TO"],)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph = Neo4jGraph()

# graph.add_graph_documents(
#     graph_documents,
#     baseEntityLabel=True,
#     include_source=True
# )

In [21]:
def show_graph():
    uri = os.environ.get("NEO4J_URI")
    auth = (os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD"))
    driver = GraphDatabase.driver(uri, auth=auth)
    session = driver.session()
    widget = GraphWidget(graph=session.run("MATCH (s) - [r:!MENTIONS] -> (t) RETURN s, r, t").graph())
    widget.node_label_mapping = 'id'
    return widget

show_graph()

GraphWidget(layout=Layout(height='800px', width='100%'))